In [1]:
%load_ext autoreload


In [ ]:
__author__ = 'biherrera'
__email__ = 'biherrera@uc.cl'

%autoreload

import sys
import os

# The following line adds the directory to the path in order to cross-reference
# files in the repo
sys.path.append(os.getcwd()[:-6])
print("Relative path: ", os.getcwd()[:-6])
import dolfin
from src.model import PerfusionGasExchangeModel
from src.params import params

print("Imported src files")
print("Starting...")
folder = "rve_40_vol"
path = os.path.join("../local-results-data", folder)
print(path)
model = PerfusionGasExchangeModel(folder_path=path, params=params)
print("Model initialised")
model.import_mesh(
    os.path.join("../raw-data/rve_40_vol", "rve_40_flat.xdmf"), type="xdmf", 
    periodic=False
)
print("Mesh imported")
model.mesh = dolfin.refine(model.mesh)
print("Mesh refined")
print("Starting (P) simulation")
model.sim_p(save=True, meshtype="tkd")
print("(P) simulation done")
# print("Starting (T) simulation")
# x = model.sim_t(hb=False, save=True)
# print("Finished (linear) guess generation")
# solution = model.sim_t(hb=True, save=True, guess=x)
# print("Done")

Relative path:  /home/bnherrerac/alveolar-perfusion-transport-modeling
Imported src files
Starting...
../local-results-data/rve_40_vol
Path exists
Model initialised
Mesh imported
Coordinates have shape  (15877, 3)
self.dir_max_flow =  40.6758673128
self.dir_min_flow =  -0.6561989643999999
self.dir_max_y =  40.6559837499
self.dir_min_y =  -0.6702492376
self.dir_max_z =  40.6433994556
self.dir_min_z =  -0.6591004764
Mesh imported
Mesh refined
Starting (P) simulation
began sim_p
Instancing boundaries
gamma_in.dir_min =  -0.6561989643999999
gamma_in.dir_max =  40.6758673128
gamma_out.dir_min =  -0.6561989643999999
gamma_out.dir_max =  40.6758673128
gamma_air.dir_min_y =  -0.6702492376
gamma_air.dir_max_y =  40.6559837499
gamma_air.dir_min_z =  -0.6591004764
gamma_air.dir_max_z =  40.6433994556
